In [1]:
using Pkg

Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\ayats\Documents\aeer-parsing`


In [3]:
import CSV as CSV
import DataFrames: DataFrame

CSV.read("./data/ORN0901WD.csv", DataFrame)

Row,2009;1;2009-1-1,00:00:00;0;2.241;328;24.64;3.571;322.1;25.54;
,String31,String
1,2009;1;2009-1-1,00:10:00;10;2.386;351.4;24.29;3.29;342;24.96;
2,2009;1;2009-1-1,00:20:00;20;1.907;355.1;24.19;2.996;341.3;25.07;
3,2009;1;2009-1-1,00:30:00;30;1.113;84.9;24.24;1.613;0.517;25.16;
4,2009;1;2009-1-1,00:40:00;40;4.031;173.4;24.76;4.694;177.7;25.18;
5,2009;1;2009-1-1,00:50:00;50;1.185;241.9;25.97;1.447;198.3;25.98;
6,2009;1;2009-1-1,01:00:00;60;1.727;56.83;25.16;2.488;90.4;25.04;
7,2009;1;2009-1-1,01:10:00;70;2.053;32.83;25.48;1.88;49.17;25.78;
8,2009;1;2009-1-1,01:20:00;80;1.76;27.14;25.32;2.135;47.11;25.96;
9,2009;1;2009-1-1,01:30:00;90;2.207;353.9;25.81;2.958;358.2;26.39;
